### Import Functions

In [9]:
import pandas as pd
import numpy as np
import csv
import glob
import os

def Grouper(df,column,criteria):
    column = column
    grouped = df.groupby(df[column])
    g = grouped.get_group(criteria)
    return g

currentweek = 14
path = r'data\2020\MasterDF\\'+str(currentweek)
try:
    os.mkdir(path)
except:
    print("Path Already Exists")

print("Import Complete")

Path Already Exists
Import Complete


### Modules for Loading and Parsing Data

In [10]:
def LoadDF(file):
    with open(file) as f:
        reader = csv.reader(f)
        data = [row for row in reader]
        
    headers = data.pop(0)
    DF = pd.DataFrame(data,columns=headers)
    return DF

def ConvertHeaders(headers):
    newheaders = []
    newheaders.append(headers[0])
    labels = ['Pass','Rec','Rush']
    x = 1
    y = 0
    z = 1
    while x < len(headers):
        newlabel = str(headers[x]+labels[y])
        if newlabel == 'YdsPass':
            newlabel += str(z)
            z += 1
        newheaders.append(newlabel)
        if headers[x] == '1D':
            y += 1
        x += 1
    return newheaders

def LoadPerfPlayerData():
    masterlist = []
    for file in glob.glob(r'data\2020\performance\OffensePlayer\*'):
        w = file.split("\\")[-1].split(".")[0].replace("OffensePlayerWeek","")
        df = pd.read_excel(file)
        headers = df.iloc[0].to_list()
        df = df.iloc[1:]
        df.columns = ConvertHeaders(headers)
        df['Week'] = int(w)
        df['SkYdsPass'] = df['YdsPass2']
        df.drop(['YdsPass2'],axis=1)
        masterlist.append(df)
    perfdata = pd.concat(masterlist)
    return perfdata

def LoadPerfTeamData(l,a):
    masterlist = []
    for file in glob.glob(r'data\2020\performance\\'+l+'\*'):
        w = file.split("\\")[-1].split(".")[0].replace(str(l+"Week"),"")
        df = pd.read_excel(file,header=None)
        headers = [str(h) + a for h in df.iloc[1].to_list()]
        newheaders = []
        x = 0
        for h in headers:
            if h not in newheaders:
                newheaders.append(h)
            else:
                x += 1
                newheaders.append(h+str(x))
        df = df.iloc[2:]
        df.columns = newheaders
        df['Week'] = int(w)
        masterlist.append(df)
    perfdata = pd.concat(masterlist)
    return perfdata

def CreatePerfTeamData():
    PerfOffenseData = LoadPerfTeamData("Offense","OFNS")
    PerfDefenseData = LoadPerfTeamData("Defense","DFNS")
    PerfAllScoreData = LoadPerfTeamData("AllScore","ALLSCR")
    PerfDSTScoreData = LoadPerfTeamData("DSTScore","DSTSCR")
    PerfTeamData = PerfOffenseData.merge(PerfDefenseData, left_on=['TmOFNS','Week'],right_on=['TmDFNS','Week'],how='inner')
    PerfTeamData = PerfTeamData.merge(PerfAllScoreData, left_on=['TmOFNS','Week'],right_on=['TmALLSCR','Week'],how='inner')
    PerfTeamData = PerfTeamData.merge(PerfDSTScoreData, left_on=['TmOFNS','Week'],right_on=['TmDSTSCR','Week'],how='inner')
    PerfTeamData = PerfTeamData.fillna(0)
    PerfTeamData['SftyALLSCR'] = PerfTeamData['SftyALLSCR'].astype(int)
    PerfTeamData['PtsFor'] = PerfTeamData['TDALLSCR'] * 6 + PerfTeamData['XPMALLSCR'] + PerfTeamData['FGMALLSCR'] * 3 + PerfTeamData['2PMALLSCR'] * 2
    PerfPtsDf = PerfTeamData[['OppALLSCR','PtsFor','Week']]
    PerfPtsDf['PtsAgainst'] = PerfPtsDf['PtsFor']
    PerfPtsDf['Opponent'] = PerfPtsDf['OppALLSCR']
    PerfPtsDf = PerfPtsDf.drop(['PtsFor','OppALLSCR'], axis=1)
    PerfTeamData = PerfTeamData.merge(PerfPtsDf, left_on=['TmOFNS','Week'],right_on=['Opponent','Week'],how='inner')
    PerfTeamData['Opponent'] = PerfTeamData['OppALLSCR']
    
    return PerfTeamData

print("Modules Loaded")

Modules Loaded


### Load Data

In [11]:
PlayerMatch = LoadDF('data/2020/Match/PlayerMatch.csv')
print(PlayerMatch.shape)
TeamMatch = LoadDF('data/2020/Match/TeamMatch.csv')
print(TeamMatch.shape)
DKPoints = LoadDF('data/2020/Performance/DKPoints.csv')
print(DKPoints.shape)
DKPointsDefense = Grouper(DKPoints,'Pos','Def')
print(DKPointsDefense.shape)
PerfPlayerData = LoadPerfPlayerData()
print(PerfPlayerData.shape)
PerfTeamData = CreatePerfTeamData()
print(PerfTeamData.shape)

(873, 6)
(32, 8)
(5639, 11)
(414, 11)
(4161, 25)
(406, 78)


C:\Users\KWRS\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\KWRS\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Performance Data

In [12]:
def CreateAverageColumns(df1,df,index,avgcols,label):
    maxweek = int(df['Week'].max())
    NewCALCAVGColumns = index
    for col in avgcols:
        newcol = str('CALCAVG'+label+col)
        df[newcol] = df[col].apply(pd.to_numeric)
        NewCALCAVGColumns.append(newcol)
    CALCAVG = df[NewCALCAVGColumns]
    rolling_avg = []
    for i in range(1,maxweek+3):
        Wlist = []
        if i == 1:
            pass
        else:
            for w in range(1,i):
                try:
                    group = Grouper(CALCAVG,'Week',w)
                    Wlist.append(group)
                except:
                    pass
            W = pd.concat(Wlist)
        
            Avg = W.groupby(index[0]).mean()
            Avg['Week'] = i
            rolling_avg.append(Avg)
    CALCAVGFulldf = pd.concat(rolling_avg).reset_index()
    return CALCAVGFulldf

print("CALCAVG Loaded")

CALCAVG Loaded


In [13]:
index = ['Player','Week']
remcols = []
cols = [col for col in PerfPlayerData.columns if col not in index and col not in remcols]
PPDOFFCALCAVGFull = CreateAverageColumns(LoadPerfPlayerData(),LoadPerfPlayerData(),index,cols,'PPDOFF')
print(PPDOFFCALCAVGFull.shape)
index = ['TmOFNS','Week']
remcols = ['Opponent','TmDSTSCR','YearDSTSCR','DateDSTSCR','TimeDSTSCR','LTimeDSTSCR','nanDSTSCR','OppDSTSCR','WeekDSTSCR','G#DSTSCR',
           'DayDSTSCR','ResultDSTSCR','TmALLSCR','YearALLSCR','DateALLSCR','TimeALLSCR','LTimeALLSCR','nanALLSCR','OppALLSCR',
           'WeekALLSCR','G#ALLSCR','DayALLSCR','ResultALLSCR','TmDFNS','GOFNS','OTALLSCR','OTDSTSCR']
cols = [col for col in PerfTeamData.columns if col not in index and col not in remcols]
PTDOFFCALCAVGFull = CreateAverageColumns(CreatePerfTeamData(),CreatePerfTeamData(),index,cols,'PTDOFF')
print(PTDOFFCALCAVGFull.shape)
index = ['Opponent','Week']
remcols = ['TmOFNS','TmDSTSCR','YearDSTSCR','DateDSTSCR','TimeDSTSCR','LTimeDSTSCR','nanDSTSCR','OppDSTSCR','WeekDSTSCR','G#DSTSCR',
           'DayDSTSCR','ResultDSTSCR','TmALLSCR','YearALLSCR','DateALLSCR','TimeALLSCR','LTimeALLSCR','nanALLSCR','OppALLSCR',
           'WeekALLSCR','G#ALLSCR','DayALLSCR','ResultALLSCR','TmDFNS','GOFNS','OTALLSCR','OTDSTSCR']
cols = [col for col in PerfTeamData.columns if col not in index and col not in remcols]
PTDOPPCALCAVGFull = CreateAverageColumns(CreatePerfTeamData(),CreatePerfTeamData(),index,cols,'PTDOPP')
print(PTDOPPCALCAVGFull.shape)

(7501, 25)


C:\Users\KWRS\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\KWRS\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(480, 51)
(480, 51)


In [14]:
#MatchFiles
PlayerMatch = LoadDF('data/2020/Match/PlayerMatch.csv')
#print(PlayerMatch.head())
TeamMatch = LoadDF('data/2020/Match/TeamMatch.csv')
#print(TeamMatch.head())

#OffenseTrain
DKPoints = LoadDF('data/2020/Performance/DKPoints.csv')
DKPoints['Week'] = DKPoints['Week'].astype(int)
DKPoints['Name'] = DKPoints['Name'].replace(dict(zip(PlayerMatch.DKPointsName, PlayerMatch.UniquePlayer)))

PerfPlayerData['Player'] = PerfPlayerData['Player'].replace(dict(zip(PlayerMatch.PerfName, PlayerMatch.UniquePlayer)))
DKPoints = DKPoints.merge(PerfPlayerData, left_on=['Name','Week'], right_on=['Player','Week'],how='inner')
PPDOFFCALCAVGFull['Player'] = PPDOFFCALCAVGFull['Player'].replace(dict(zip(PlayerMatch.PerfName, PlayerMatch.UniquePlayer)))
DKPoints = DKPoints.merge(PPDOFFCALCAVGFull, left_on=['Name','Week'], right_on=['Player','Week'],how='inner')
DKPoints['OffenseUID'] = DKPoints['Team'].replace(dict(zip(TeamMatch.DKAbbrev, TeamMatch.UID)))
PTDOFFCALCAVGFull['OffenseUID'] = PTDOFFCALCAVGFull['TmOFNS'].replace(dict(zip(TeamMatch.PerfMatch, TeamMatch.UID)))
DKPoints = DKPoints.merge(PTDOFFCALCAVGFull, left_on=['OffenseUID','Week'], right_on=['OffenseUID','Week'],how='inner')
DKPoints['DefenseUID'] = DKPoints['Oppt'].replace(dict(zip(TeamMatch.DKAbbrev, TeamMatch.UID)))
PTDOPPCALCAVGFull['DefenseUID'] = PTDOPPCALCAVGFull['Opponent'].replace(dict(zip(TeamMatch.PerfMatch, TeamMatch.UID)))
DKPoints = DKPoints.merge(PTDOPPCALCAVGFull, left_on=['DefenseUID','Week'], right_on=['DefenseUID','Week'],how='inner')

print(DKPoints.shape)
DKPoints.to_csv(path+'\\PlayerTrain.csv')

#DefenseTrain
DKPoints = LoadDF('data/2020/Performance/DKPoints.csv')
DKPoints['Week'] = DKPoints['Week'].astype(int)
DKPointsDefense = Grouper(DKPoints,'Pos','Def')
DKPointsDefense['OffenseUID'] = DKPointsDefense['Team'].replace(dict(zip(TeamMatch.DKAbbrev, TeamMatch.UID)))
DKPointsDefense = DKPointsDefense.merge(PTDOFFCALCAVGFull, left_on=['OffenseUID','Week'], right_on=['OffenseUID','Week'],how='inner')
DKPointsDefense['DefenseUID'] = DKPointsDefense['Oppt'].replace(dict(zip(TeamMatch.DKAbbrev, TeamMatch.UID)))
DKPointsDefense = DKPointsDefense.merge(PTDOPPCALCAVGFull, left_on=['DefenseUID','Week'], right_on=['DefenseUID','Week'],how='inner')

print(DKPointsDefense.shape)
DKPointsDefense.to_csv(path+'\\DefenseTrain.csv')

(2655, 161)


C:\Users\KWRS\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(382, 113)


In [15]:
print(TeamMatch.head())

        Name        ID    UID TeamAbbrev PerfMatch OpposingTeamAbbrev  \
0  Steelers   15260860  10001        PIT       PIT                PIT   
1     Bills   15260861  10002        BUF       BUF                BUF   
2    Eagles   15260862  10003        PHI       PHI                PHI   
3     49ers   15260863  10004         SF       SFO                 SF   
4     Bears   15260864  10005        CHI       CHI                CHI   

          DKTeam DKAbbrev  
0     Pittsburgh      pit  
1        Buffalo      buf  
2   Philadelphia      phi  
3  San Francisco      sfo  
4        Chicago      chi  


In [16]:
def CreateProjectionMDF(file):
    PlayerMatch = LoadDF('data/2020/Match/PlayerMatch.csv')
    TeamMatch = LoadDF('data/2020/Match/TeamMatch.csv')

    w = file.split("/")[-1].split(".")[0].replace("Week","")
    dk = pd.read_csv(file)
    dk['HomeTeam'] = dk['Game Info'].str.split(' ').str[0].str.split('@').str[1]
    dk['AwayTeam'] = dk['Game Info'].str.split(' ').str[0].str.split('@').str[0]
    dk.loc[(dk['HomeTeam'] == dk['TeamAbbrev']), 'OpposingTeam'] = dk['AwayTeam']
    dk.loc[(dk['AwayTeam'] == dk['TeamAbbrev']), 'OpposingTeam'] = dk['HomeTeam']
    dk['OpponentUID'] = dk['OpposingTeam'].replace(dict(zip(TeamMatch.TeamAbbrev, TeamMatch.UID)))
    dk['TeamUID'] = dk['TeamAbbrev'].replace(dict(zip(TeamMatch.TeamAbbrev, TeamMatch.UID)))
    dk['OffenseTeam'] = dk['TeamAbbrev'].replace(dict(zip(TeamMatch.TeamAbbrev, TeamMatch.ID)))
    dk['Week'] = int(w)
    dk['UniquePlayer'] = dk['Name'] + dk['Position'] + dk['OffenseTeam']
    return dk

projfile = str('data/2020/draftkings/Week'+str(currentweek)+'.csv')
ProjMDF = CreateProjectionMDF(projfile)
ProjDefenseMDF = Grouper(ProjMDF,'Position','DST')
ProjMDF = ProjMDF.merge(PTDOFFCALCAVGFull, left_on=['TeamUID','Week'], right_on=['OffenseUID','Week'],how='inner')
ProjMDF = ProjMDF.merge(PTDOPPCALCAVGFull, left_on=['OpponentUID','Week'], right_on=['DefenseUID','Week'],how='inner')
ProjMDF = ProjMDF.merge(PPDOFFCALCAVGFull, left_on=['UniquePlayer','Week'], right_on=['Player','Week'],how='inner')
print(ProjMDF.shape)
ProjMDF.to_csv(path+'\\PlayerProject.csv')


ProjDefenseMDF = ProjDefenseMDF.merge(PTDOPPCALCAVGFull, left_on=['TeamUID','Week'], right_on=['DefenseUID','Week'],how='inner')
ProjDefenseMDF = ProjDefenseMDF.merge(PTDOFFCALCAVGFull, left_on=['OpponentUID','Week'], right_on=['OffenseUID','Week'],how='inner')
print(ProjDefenseMDF.shape)
ProjDefenseMDF.to_csv(path+'\\DefenseProject.csv')

print("Data Preparation Complete")

(291, 143)
(32, 119)


### DraftKings Data